<a href="https://colab.research.google.com/github/Gayeon6423/2023BigContest/blob/main/Modeling_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
from sklearn.impute import KNNImputer
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

pd.set_option('display.max_seq_items',None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

In [7]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [8]:
mountInstance = LoadGoogleDriveData()

In [62]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

# MARKET_2022_DATA = mountInstance.loadData(
#     file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2022_GROUPED_EXPAND.csv",
#     unicode = "euc-kr", columnTF = False).drop_duplicates()

# MARKET_2023_DATA = mountInstance.loadData(
#     file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2023_GROUPED_EXPAND.csv",
#     unicode = "euc-kr", columnTF = False).drop_duplicates()

# KCD_2022_2023 = mountInstance.loadData(
#     file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "KCD_DATA.csv",
#     unicode = "euc-kr", columnTF = False)

KCD_MARKET_CONTEST_JOIN = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/", file_name_extension = "KCD_MARKET_CONTEST_JOIN.csv",
    unicode = "utf-8-sig", columnTF = False)

KCD_SEASON_PATTERN = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/", file_name_extension = "KCD_SEASON_PATTERN.csv",
    unicode = "utf-8-sig", columnTF = False)


### 1. 이상치 처리

In [63]:
# 필요없는 칼럼 삭제 및 칼럼 정리
KCD_MARKET_CONTEST_JOIN = KCD_MARKET_CONTEST_JOIN.drop(['기준_분기_코드','기준_년_코드','상권_식별자','상권명','외식업종_중분류','기준분기','상권_코드_명',],axis=1)
KCD_MARKET_CONTEST_JOIN['기준_년_코드'] = KCD_MARKET_CONTEST_JOIN['기준일자'].str[:4].astype('int64')
KCD_MARKET_CONTEST_JOIN['기준_분기_코드'] = KCD_MARKET_CONTEST_JOIN['기준일자'].str[4:5].astype('int64')
KCD_MARKET_CONTEST_JOIN = KCD_MARKET_CONTEST_JOIN.drop(['기준일자'],axis=1)

In [64]:
# 이상치 처리 함수
def preprocess_outlier(dataframe):
    for column in dataframe.columns:
        column_data = dataframe[column]

        # 1사분위수와 3사분위수 계산
        q1 = column_data.quantile(0.25)
        q3 = column_data.quantile(0.75)

        # IQR 계산
        iqr = q3 - q1

        # 이상치 경계값 계산
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        # 이상치를 NaN으로 대체
        dataframe[column] = dataframe[column].apply(
            lambda x: x if lower_bound <= x <= upper_bound else None)

    return dataframe

In [65]:
preprocess_outlier(KCD_MARKET_CONTEST_JOIN)

,상권코드,STDG_EMD_CD,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,사업장임대면적_변동계수,...,토요일_생활인구_수,일요일_생활인구_수,상권_코드_2022,상권_코드_2023,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,기준_년_코드,기준_분기_코드
0,2120098.0000,11440127.0000,6.6481,7.9662,1.6407,17.8384,2.4877,3.9457,2.6117,1.2383,...,27161.0000,24683.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1
1,2120098.0000,11440127.0000,NaN,NaN,0.9338,12.4025,1.6792,3.0789,0.9359,1.2653,...,27161.0000,24683.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1
2,2120234.0000,11710107.0000,5.8697,6.1537,1.2814,14.1810,6.4333,4.2607,1.5821,1.4835,...,195928.0000,180363.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1
3,2120186.0000,11680107.0000,7.3501,7.3485,2.5290,10.4700,3.1739,NaN,2.5521,1.5978,...,274353.0000,235313.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1
4,2120186.0000,11680107.0000,7.1350,7.1350,1.1558,19.9781,3.1739,1.3306,1.9152,1.8144,...,274353.0000,235313.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3359,NaN,11380110.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1
3360,NaN,11380101.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1
3361,NaN,11200109.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1
3362,NaN,11170132.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,1


### 2. 결측치 처리

In [66]:
# 결측치 처리할 칼럼 리스트
num_col = KCD_MARKET_CONTEST_JOIN.columns.tolist()
num_col.remove('기준_년_코드')
num_col.remove('기준_분기_코드')

# KNNImputer로 결측치 대체
imputer = KNNImputer(n_neighbors=5)
KCD_MARKET_CONTEST_JOIN[num_col] = imputer.fit_transform(KCD_MARKET_CONTEST_JOIN[num_col])

KCD_MARKET_CONTEST_JOIN[KCD_MARKET_CONTEST_JOIN.isna().any(axis=1)]

,상권코드,STDG_EMD_CD,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,사업장임대면적_변동계수,...,토요일_생활인구_수,일요일_생활인구_수,상권_코드_2022,상권_코드_2023,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,기준_년_코드,기준_분기_코드


In [68]:
# 분기별 상권데이터를 소호 신용데이터를 활용하여 월별로 확장(계절성 지수)
## 기준_분기_코드 칼럼의 값을 변경하여 월 칼럼 생성
KCD_MARKET_CONTEST_JOIN['월'] = KCD_MARKET_CONTEST_JOIN['기준_분기_코드'].apply(lambda x: '1 2 3' if x == 1 else ('4 5 6' if x == 2 else ('7 8 9' if x == 3 else ('10 11 12' if x == 4 else x))))

## '월' 칼럼의 값을 공백을 기준으로 분리
KCD_MARKET_CONTEST_JOIN['월'] = KCD_MARKET_CONTEST_JOIN['월'].str.split()

## '월' 칼럼을 분리하기 위해 데이터프레임을 재구성
tmp = pd.DataFrame({'기준_월_코드': np.concatenate(KCD_MARKET_CONTEST_JOIN['월'].values)})

## 데이터를 인덱스 기준으로 확장
EXPAND_MON = KCD_MARKET_CONTEST_JOIN.loc[KCD_MARKET_CONTEST_JOIN.index.repeat(3)].reset_index(drop=True)
EXPAND_MON = EXPAND_MON.drop(columns = '월')

## 확장된 데이터 병합
KCD_MARKET_CONTEST_JOIN_MON = pd.concat([EXPAND_MON,tmp],axis=1)

## DATA_CRTR_YM 칼럼 생성(기준년도+기준월)
KCD_MARKET_CONTEST_JOIN_MON['기준_월_코드'] = KCD_MARKET_CONTEST_JOIN_MON['기준_월_코드'].apply(lambda x : '0' + x if len(x) == 1 else x)

KCD_MARKET_CONTEST_JOIN_MON['DATA_CRTR_YM'] = KCD_MARKET_CONTEST_JOIN_MON['기준_년_코드'].astype('str') + KCD_MARKET_CONTEST_JOIN_MON['기준_월_코드'].astype('str')
KCD_MARKET_CONTEST_JOIN_MON['DATA_CRTR_YM'] = KCD_MARKET_CONTEST_JOIN_MON['DATA_CRTR_YM'].astype('int64')
KCD_MARKET_CONTEST_JOIN_MON

,상권코드,STDG_EMD_CD,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,사업장임대면적_변동계수,...,상권_코드_2022,상권_코드_2023,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,기준_년_코드,기준_분기_코드,기준_월_코드,DATA_CRTR_YM
0,2120098.0000,11440127.0000,6.6481,7.9662,1.6407,17.8384,2.4877,3.9457,2.6117,1.2383,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1,01,202201
1,2120098.0000,11440127.0000,6.6481,7.9662,1.6407,17.8384,2.4877,3.9457,2.6117,1.2383,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1,02,202202
2,2120098.0000,11440127.0000,6.6481,7.9662,1.6407,17.8384,2.4877,3.9457,2.6117,1.2383,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1,03,202203
3,2120098.0000,11440127.0000,6.5544,7.4480,0.9338,12.4025,1.6792,3.0789,0.9359,1.2653,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1,01,202201
4,2120098.0000,11440127.0000,6.5544,7.4480,0.9338,12.4025,1.6792,3.0789,0.9359,1.2653,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2022,1,02,202202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10087,2320041.0000,11170132.0000,5.5445,6.5609,1.5251,28.4730,1.8065,2.9825,1.5317,2.1408,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2023,1,02,202302
10088,2320041.0000,11170132.0000,5.5445,6.5609,1.5251,28.4730,1.8065,2.9825,1.5317,2.1408,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2023,1,03,202303
10089,2520041.0000,11170119.0000,5.9212,6.8389,1.9219,28.0888,2.1806,3.0449,2.0767,2.1593,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2023,1,01,202301
10090,2520041.0000,11170119.0000,5.9212,6.8389,1.9219,28.0888,2.1806,3.0449,2.0767,2.1593,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2023,1,02,202302


In [ ]:
CONTEST_2023.head(5)

,LT_UNQ_NO,INDUSTRY_CD,STDG_EMD_CD,SLS_GRD_2301,SLS_GRD_2302
0,1168010800102090000,A01,11680108,NaN,NaN
1,1135010300106330015,A05,11350103,NaN,NaN
2,1154510200109830004,A02,11545102,NaN,NaN
3,1162010200100920362,A01,11620102,NaN,NaN
4,1168011000104580000,A03,11680110,NaN,NaN
